In [ ]:
!wget https://digix-algo-challenge.obs.cn-east-2.myhuaweicloud.com/2020/cv/6rKDTsB6sX8A1O2DA2IAq7TgHPdSPxJF/train_data.zip

In [ ]:
!unzip train_data.zip

In [ ]:
import torch
import torch.nn as nn

In [ ]:
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.utils.data import DataLoader
import time 
import numpy as np

In [ ]:
resnet50 = models.resnet50(pretrained=True)

In [ ]:
count = 0
for param in resnet50.parameters():
    count += 1
    if count < 100:
      param.requires_grad = False
    else:
      param.requires_grad = True

In [ ]:
#avgpool_inputs = resnet50.avgpool.in_features
resnet50.fc = nn.Sequential(
)

In [ ]:
resnet50.cuda()

In [ ]:
resnet50 = resnet50.to('cuda:0')

In [ ]:
def sigmoid(tensor, temp=1.0):
    """ temperature controlled sigmoid
    takes as input a torch tensor (tensor) and passes it through a sigmoid, controlled by temperature: temp
    """
    exponent = -tensor / temp
    # clamp the input tensor for stability
    exponent = torch.clamp(exponent, min=-50, max=50)
    y = 1.0 / (1.0 + torch.exp(exponent))
    return y


def compute_aff(x):
    """computes the affinity matrix between an input vector and itself"""
    return torch.mm(x, x.t())


class SmoothAP(torch.nn.Module):
    """PyTorch implementation of the Smooth-AP loss.
    implementation of the Smooth-AP loss. Takes as input the mini-batch of CNN-produced feature embeddings and returns
    the value of the Smooth-AP loss. The mini-batch must be formed of a defined number of classes. Each class must
    have the same number of instances represented in the mini-batch and must be ordered sequentially by class.
    e.g. the labels for a mini-batch with batch size 9, and 3 represented classes (A,B,C) must look like:
        labels = ( A, A, A, B, B, B, C, C, C)
    (the order of the classes however does not matter)
    For each instance in the mini-batch, the loss computes the Smooth-AP when it is used as the query and the rest of the
    mini-batch is used as the retrieval set. The positive set is formed of the other instances in the batch from the
    same class. The loss returns the average Smooth-AP across all instances in the mini-batch.
    Args:
        anneal : float
            the temperature of the sigmoid that is used to smooth the ranking function. A low value of the temperature
            results in a steep sigmoid, that tightly approximates the heaviside step function in the ranking function.
        batch_size : int
            the batch size being used during training.
        num_id : int
            the number of different classes that are represented in the batch.
        feat_dims : int
            the dimension of the input feature embeddings
    Shape:
        - Input (preds): (batch_size, feat_dims) (must be a cuda torch float tensor)
        - Output: scalar
    Examples::
        >>> loss = SmoothAP(0.01, 60, 6, 256)
        >>> input = torch.randn(60, 256, requires_grad=True).cuda()
        >>> output = loss(input)
        >>> output.backward()
    """

    def __init__(self, anneal, batch_size, num_id, feat_dims):
        """
        Parameters
        ----------
        anneal : float
            the temperature of the sigmoid that is used to smooth the ranking function
        batch_size : int
            the batch size being used
        num_id : int
            the number of different classes that are represented in the batch
        feat_dims : int
            the dimension of the input feature embeddings
        """
        super(SmoothAP, self).__init__()

        assert(batch_size%num_id==0)

        self.anneal = anneal
        self.batch_size = batch_size
        self.num_id = num_id
        self.feat_dims = feat_dims

    def forward(self, preds):
        """Forward pass for all input predictions: preds - (batch_size x feat_dims) """


        # ------ differentiable ranking of all retrieval set ------
        # compute the mask which ignores the relevance score of the query to itself
        mask = 1.0 - torch.eye(self.batch_size) 
        mask = mask.unsqueeze(dim=0).repeat(self.batch_size, 1, 1)
        # compute the relevance scores via cosine similarity of the CNN-produced embedding vectors
        sim_all = compute_aff(preds)
        sim_all_repeat = sim_all.unsqueeze(dim=1).repeat(1, self.batch_size, 1)
        # compute the difference matrix
        sim_diff = sim_all_repeat - sim_all_repeat.permute(0, 2, 1)
        # pass through the sigmoid
        sim_sg = sigmoid(sim_diff, temp=self.anneal) * mask.cuda()
        # compute the rankings
        sim_all_rk = torch.sum(sim_sg, dim=-1) + 1

        # ------ differentiable ranking of only positive set in retrieval set ------
        # compute the mask which only gives non-zero weights to the positive set
        xs = preds.view(self.num_id, int(self.batch_size / self.num_id), self.feat_dims)
        pos_mask = 1.0 - torch.eye(int(self.batch_size / self.num_id))
        pos_mask = pos_mask.unsqueeze(dim=0).unsqueeze(dim=0).repeat(self.num_id, int(self.batch_size / self.num_id), 1, 1)
        # compute the relevance scores
        sim_pos = torch.bmm(xs, xs.permute(0, 2, 1))
        sim_pos_repeat = sim_pos.unsqueeze(dim=2).repeat(1, 1, int(self.batch_size / self.num_id), 1)
        # compute the difference matrix
        sim_pos_diff = sim_pos_repeat - sim_pos_repeat.permute(0, 1, 3, 2)
        # pass through the sigmoid
        sim_pos_sg = sigmoid(sim_pos_diff, temp=self.anneal) * pos_mask.cuda()
        # compute the rankings of the positive set
        sim_pos_rk = torch.sum(sim_pos_sg, dim=-1) + 1

        # sum the values of the Smooth-AP for all instances in the mini-batch
        ap = torch.zeros(1).cuda()
        group = int(self.batch_size / self.num_id)
        for ind in range(self.num_id):
            pos_divide = torch.sum(sim_pos_rk[ind] / (sim_all_rk[(ind * group):((ind + 1) * group), (ind * group):((ind + 1) * group)]))
            ap = ap + ((pos_divide / group) / self.batch_size)

        return (1-ap)


In [ ]:
loss_func = SmoothAP(0.01, 120, 12, 2048)
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, resnet50.parameters()),lr=0.005)

In [ ]:
import random
import os
from torch.utils.data import DataLoader, Sampler, Dataset
from torchvision import datasets,transforms
from keras.preprocessing import image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
class customData(Dataset):
    def __init__(self, img_path, dataset = 'train'):
      self.img_name = img_path
      self.dataset = dataset
      self.len = len(img_path)

    def __len__(self):
      return len(self.img_name)

    def __getitem__(self, item):
      index = item % self.len
      # print("i={},index={}".format(i, index))
      image_path = self.img_name[index]
      img = Image.open(image_path)
      img = transforms.RandomResizedCrop(224)(img)
      img = transforms.ToTensor()(img)
      if img.shape[0] == 1:
        img = img.numpy()
        img = np.concatenate((img, img, img))
        img = torch.from_numpy(img)               
      img = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(img)
      return img
    
#####################以上为图像数据读取，返回（img, label）#########################


In [ ]:
from tqdm import tqdm

In [ ]:
def train_and_valid(model, loss_function, optimizer, file_path = 'train_data', num_class = 12, num_data = 10, epochs=10):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    history = []
    best_acc = 0.0
    best_epoch = 0
 
    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
 
        model.train()
 
        train_loss = 0.0
        valid_loss = 0.0

        dirlist = os.listdir('train_data')
        dirlist.remove('label.txt')
        
        for i in tqdm(range(400)):
            batch_list = []
            num_count = 0
            for classlabel in random.sample(dirlist,24):
                if num_count <= 12:
                  imglist = os.listdir(file_path + '/' + classlabel)
                  try:
                    sample = random.sample(imglist,10)
                    num_count += 1
                    for i in sample:
                      batch_list.append(file_path + '/' + classlabel + '/' + i)
                  except:
                    pass

            image_datasets = customData(img_path=batch_list) 
            traindataloaders = torch.utils.data.DataLoader(image_datasets,batch_size=120,shuffle=False) 
          
            for i, input in enumerate(traindataloaders):
              inputs = input.cuda()
                #因为这里梯度是累加的，所以每次记得清零
              optimizer.zero_grad()
 
              outputs = model(inputs)

              loss = loss_function(outputs)
              print(loss)
              loss.backward()
 
              optimizer.step()
 
              train_loss += loss * inputs.size(0)
              break

        with torch.no_grad():
            model.eval()
            for i in range(40):
                valid_list = []
                for classlabel in random.sample(dirlist,24):
                  if num_count <= 12:
                    imglist = os.listdir(file_path + '/' + classlabel)
                    try:
                      sample = random.sample(imglist,10)
                      num_count += 1
                      for i in sample:
                        valid_list.append(file_path + '/' + classlabel + '/' + i)
                    except:
                      pass
                image_datasets = customData(img_path=valid_list) 
                validdataloaders = torch.utils.data.DataLoader(image_datasets,batch_size=120,
                                                               shuffle=False)
                for input in validdataloaders:
                  inputs = input.cuda()
                  outputs = model(inputs)
 
                  loss = loss_function(outputs)
 
                  valid_loss += loss * inputs.size(0)
                  break  

        avg_train_loss = train_loss/400
        
 
        avg_valid_loss = valid_loss/40
        
 
        history.append([avg_train_loss, avg_valid_loss])
 
        
 
        epoch_end = time.time()
 
        print("Epoch: {:03d}, Training: Loss: {:.4f}, \n\t\tValidation: Loss: {:.4f}, Time: {:.4f}s".format(
            epoch+1, avg_valid_loss, avg_valid_loss, epoch_end-epoch_start
        ))
        
 
        torch.save(model, '/content/drive/Shared drives/my_new_pan/' + str(epoch+1)+'.pt')
    return model, history

In [ ]:
trained_model, history = train_and_valid(resnet50, loss_func, optimizer)